In [3]:
import openai

# 配置 OpenAI API
openai.api_base = "http://172.16.3.115:2024/v1"
openai.api_key = "sk-CFvfPTt9gypWFMsY0mWSd29mysqqqqqqqqqqqq111"

def analyze_ticket(commands_in_ticket, executed_commands):
    """
    使用 OpenAI API 分析工单命令与实际执行命令的匹配情况。
    
    参数:
    commands_in_ticket (str): 工单内容中的命令列表（文本形式）。
    executed_commands (list): 实际执行的命令集合（列表形式）。
    
    返回:
    dict: 包括结果分析、不一致项和改进建议。
    """
    # 构造提示词
    prompt = f"""
    工单命令内容如下：
    {commands_in_ticket}

    实际执行的命令如下：
    {executed_commands}

    请完成以下任务：
    1. 分析工单中的命令与实际执行命令是否完全一致。
    2. 如果不一致，指出不一致的命令和对应的差异原因。
    3. 提供改进建议，如何更好地保证命令执行符合工单要求。
    """

    # 调用 OpenAI API
    response = openai.ChatCompletion.create(
        model="qwen2.5-14b",  # 使用大模型，例如 GPT-4
        messages=[
            {"role": "system", "content": "你是一位专业的IT运维助手，善于分析命令执行的准确性并给出改进建议。"},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=500
    )

    # 获取模型返回结果
    result = response['choices'][0]['message']['content']
    return result


# 示例使用
if __name__ == "__main__":
    # 输入示例
    ticket_content = """
    1. 创建新用户： useradd alice
    2. 修改密码： passwd alice
    3. 设置用户组： usermod -aG admin alice
    4. 检查用户是否创建： id alice
    """
    executed_commands = [
        "useradd alice",
        "passwd alice",
        "usermod -aG admin alice",
        "id bob"  # 这里故意写错以制造不一致
    ]

    # 调用分析函数
    analysis_result = analyze_ticket(ticket_content, executed_commands)
    print("分析结果：")
    print(analysis_result)


分析结果：
1. 分析工单中的命令与实际执行的命令对比：
   - 工单中的命令与实际执行的命令大部分一致，但是最后一条命令不同。工单中的命令是检查用户alice的信息，而实际执行的命令是检查用户bob的信息。
   
2. 不一致的命令和对应的差异原因：
   - 工单命令：`id alice`
   - 实际执行命令：`id bob`
   
   差异原因：在最后一步执行时，操作人员可能误输入了另一个用户名，导致检查的不是工单中指定的用户alice，而是检查了bob的信息。这可能是因为操作人员操作失误或混淆了操作对象。

3. 改进建议：
   - 在执行每个命令前进行确认，确保操作对象和操作内容与工单要求一致。
   - 在执行敏感操作，如创建用户、修改密码等操作时，可以先用`-D`查看默认配置，确保配置正确，再执行命令。
   - 使用脚本自动化处理，以减少人为错误。在脚本中加入必要的检查和确认步骤，比如检查用户是否存在，密码是否符合策略等。
   - 在执行完每个步骤后，通过日志或系统状态确认是否成功执行，并与预期结果对比，确保操作正确。
   - 执行完所有命令后，再次检查用户信息以确保所有操作都正确完成，可以使用`id alice`命令确认alice的用户信息，包括其组信息等。
